In [1]:
import sys
sys.path.insert(0, '..')

# Инициализируем граф

In [2]:
import pandas as pd

from src.eventstream.eventstream import Eventstream
from src.eventstream.schema import RawDataSchema, EventstreamSchema
from src.graph.p_graph import PGraph, EventsNode

raw_data = pd.read_csv('simple-onlineshop.csv')

raw_data_schema = RawDataSchema(
                            event_name="event",
                            event_timestamp="timestamp",
                            user_id="user_id")

source = Eventstream(
    raw_data=raw_data,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

graph = PGraph(source_stream=source)

In [3]:
source_df = source.to_dataframe()

## CutPathAfter

### Описание работы функции

Обрезает eventstream сразу после одного из событий, указанных в списке cutoff_events (первого в траектории). Остается часть траектории до первого найденного события из списка включительно. Само событие остается в траектории.

Параметры:
```cut_shift``` - смещение относительно обозначенного события в сторону сокращения траектории (влево)
```min_cjm``` - минимальное кол-во событий в траектории, которые надо оставить, если в траектории меньшее количество событий, то они будут удалены



### 1 событие

In [4]:
from src.data_processors_lib.rete import CutPathAfterEvent, CutPathAfterEventParams
params = {
    'cutoff_events': ['payment_done'],
    'cut_shift': 0,
    'min_cjm': 0
}

cut_after = EventsNode(
    CutPathAfterEvent(params=CutPathAfterEventParams(**params)))

graph.add_node(
    node=cut_after,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_after
)


/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_after_event.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut["rw_cumsum"] = df_cut.loc[::-1].groupby([user_col])[time_col].transform(
/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_after_event.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_del["ref"] = df_to_del[eventstream.schema.event_id]


In [5]:
df8 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df8.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [6]:
df8['event_type'].value_counts()

raw    32666
Name: event_type, dtype: int64

In [7]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [8]:
source_df[source_df['event_name'] == 'payment_done']['user_id'].nunique()

653

In [9]:
check_id = 629881394
print(len(df8[df8['user_id'] == check_id]))
event_time = source_df[(source_df['user_id'] == check_id) & \
    (source_df['event_name'] == 'payment_done')]['event_timestamp'].values[0]

26


In [10]:
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] <= event_time)]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,6a803b23-1ea7-4497-92f6-a515e7437fba,raw,7,main,2019-11-01 22:28:54.791683,629881394
9,4beaced5-47d8-426c-ab68-c7e767de8c99,raw,9,catalog,2019-11-01 22:29:01.049513,629881394
11,16d3a1fd-9cc6-490d-91eb-44d25f18991f,raw,11,catalog,2019-11-01 22:29:32.322458,629881394
13,8a34d475-66bf-41c4-b64b-912ae8192933,raw,13,catalog,2019-11-01 22:30:09.450839,629881394
14,2576ca38-e122-46e5-a0b3-aabda5e36d76,raw,14,catalog,2019-11-01 22:31:05.565762,629881394
15,16107705-9742-4cb2-a25d-ba1614f4b0bc,raw,15,main,2019-11-01 22:31:08.333560,629881394
16,57e49264-658b-4de5-ac89-56d94c97d41c,raw,16,catalog,2019-11-01 22:31:09.010626,629881394
17,852dccfc-08c5-4c76-9090-7fc37e7a43dc,raw,17,product1,2019-11-01 22:31:10.416231,629881394
18,a7fa9624-bcb9-4005-b7c5-7a168bbd88cf,raw,18,catalog,2019-11-01 22:31:43.019527,629881394
19,b7096391-1ff2-4560-a85d-9b1c35533003,raw,19,catalog,2019-11-01 22:32:01.596163,629881394


In [11]:
print(len(source_df[source_df['user_id'] == check_id]))
source_df[source_df['user_id'] == check_id]

48


,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,6a803b23-1ea7-4497-92f6-a515e7437fba,raw,7,main,2019-11-01 22:28:54.791683,629881394
9,4beaced5-47d8-426c-ab68-c7e767de8c99,raw,9,catalog,2019-11-01 22:29:01.049513,629881394
11,16d3a1fd-9cc6-490d-91eb-44d25f18991f,raw,11,catalog,2019-11-01 22:29:32.322458,629881394
13,8a34d475-66bf-41c4-b64b-912ae8192933,raw,13,catalog,2019-11-01 22:30:09.450839,629881394
14,2576ca38-e122-46e5-a0b3-aabda5e36d76,raw,14,catalog,2019-11-01 22:31:05.565762,629881394
15,16107705-9742-4cb2-a25d-ba1614f4b0bc,raw,15,main,2019-11-01 22:31:08.333560,629881394
16,57e49264-658b-4de5-ac89-56d94c97d41c,raw,16,catalog,2019-11-01 22:31:09.010626,629881394
17,852dccfc-08c5-4c76-9090-7fc37e7a43dc,raw,17,product1,2019-11-01 22:31:10.416231,629881394
18,a7fa9624-bcb9-4005-b7c5-7a168bbd88cf,raw,18,catalog,2019-11-01 22:31:43.019527,629881394
19,b7096391-1ff2-4560-a85d-9b1c35533003,raw,19,catalog,2019-11-01 22:32:01.596163,629881394


User_id = 47562303
Событие ```payment_done``` встречается дважды


In [12]:
check_id = 47562303
print(len(df8[df8['user_id'] == check_id]))
event_time = source_df[(source_df['user_id'] == check_id) & \
    (source_df['event_name'] == 'payment_done')]['event_timestamp'].min()

7


In [13]:
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] <= event_time)]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
4029,0d8a5e7f-fb77-4787-8716-7fe87da1022a,raw,4029,catalog,2019-12-29 16:55:27.897130,47562303
4030,7cfd0385-c4af-4145-9a47-1715939eea1b,raw,4030,cart,2019-12-29 16:55:41.539826,47562303
4031,9fe9f988-6215-4743-ae10-a49715fa189a,raw,4031,delivery_choice,2019-12-29 16:55:44.747151,47562303
4032,5b65e18c-1960-4e30-ac3c-2d9ad97468e5,raw,4032,delivery_courier,2019-12-29 16:55:49.061226,47562303
4033,f700e45c-3c1e-4e05-b422-9a82a3667d9d,raw,4033,payment_choice,2019-12-29 16:55:49.201853,47562303
4034,3d28cc24-bde7-47be-b1de-b74c5aaa9450,raw,4034,payment_cash,2019-12-29 16:55:52.004250,47562303
4035,f7fb76a8-de9f-4cee-ba39-3ee8d2826e49,raw,4035,payment_done,2019-12-29 16:55:52.350396,47562303


In [14]:
source_df[(source_df['event_name'] == 'payment_done') | \
          (source_df['event_name'] == 'payment_cash') ]\
    .groupby('user_id')['event_name'].nunique().sort_values(ascending=False)

user_id
168688753    2
830731696    2
849341362    2
845778461    2
652347392    2
            ..
355840584    1
358346667    1
363256580    1
365193081    1
999642905    1
Name: event_name, Length: 730, dtype: int64

In [15]:
source_df['event_name'].value_counts()

catalog             14518
main                 5635
lost                 3098
cart                 2842
product2             2172
delivery_choice      1686
product1             1515
payment_choice       1107
delivery_courier      834
payment_done          706
payment_card          565
delivery_pickup       506
payment_cash          197
Name: event_name, dtype: int64

### сдвиг

In [16]:
from src.data_processors_lib.rete import CutPathAfterEvent, CutPathAfterEventParams
params = {
    'cutoff_events': ['payment_done'],
    'cut_shift': 1,
    'min_cjm': 0
}

cut_after = EventsNode(
    CutPathAfterEvent(params=CutPathAfterEventParams(**params)))

graph.add_node(
    node=cut_after,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_after
)

/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_after_event.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut["rw_cumsum"] = df_cut.loc[::-1].groupby([user_col])[time_col].transform(
/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_after_event.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_del["ref"] = df_to_del[eventstream.schema.event_id]


In [17]:
df81 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df81.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [18]:
df81['event_type'].value_counts()

raw    28915
Name: event_type, dtype: int64

In [19]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [20]:
source_df[source_df['event_name'] == 'payment_done']['user_id'].nunique()

653

In [21]:
check_id = 629881394
print('Длина отрезанной траектории - ', len(df81[df81['user_id'] == check_id]))
df81[df81['user_id'] == check_id]

Длина отрезанной траектории -  25


,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,9863622f-3251-4e1f-a28d-2cfb9889736b,raw,7,main,2019-11-01 22:28:54.791683,629881394.0
9,3ce21b73-8cd5-445a-ac87-dca35447093c,raw,9,catalog,2019-11-01 22:29:01.049513,629881394.0
11,9e74c251-b5d9-41f5-9178-55b869221bcd,raw,11,catalog,2019-11-01 22:29:32.322458,629881394.0
13,7519f116-53fd-43a5-9408-b39aee3e2e91,raw,13,catalog,2019-11-01 22:30:09.450839,629881394.0
14,e3efa4db-637e-4f7d-97ca-47627452e66e,raw,14,catalog,2019-11-01 22:31:05.565762,629881394.0
15,33fd2fec-0669-4448-b679-a61166ddeed5,raw,15,main,2019-11-01 22:31:08.333560,629881394.0
16,13a72e04-a34e-4efc-b2bb-2b280c76498d,raw,16,catalog,2019-11-01 22:31:09.010626,629881394.0
17,4d6b8867-1543-48ed-b9a6-fffdd5f66f57,raw,17,product1,2019-11-01 22:31:10.416231,629881394.0
18,af71ac16-aca0-41df-b073-4055d2b1951a,raw,18,catalog,2019-11-01 22:31:43.019527,629881394.0
19,c0935c1a-64af-4b5e-a297-fc9b8ba15c77,raw,19,catalog,2019-11-01 22:32:01.596163,629881394.0


In [22]:
# Время, по которому обрезается траектория, здесь задаем конкретное событие payment_cash, для теста надо будет автоматизировать

event_time = source_df[(source_df['user_id'] == check_id) & \
    (source_df['event_name'] == 'payment_cash')]['event_timestamp'].values[0]

In [23]:
print('Длина обрезанной траектории из изначального датасета - ', len(source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] <= event_time)]))
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] <= event_time)]

Длина обрезанной траектории из изначального датасета -  25


,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,9863622f-3251-4e1f-a28d-2cfb9889736b,raw,7,main,2019-11-01 22:28:54.791683,629881394
9,3ce21b73-8cd5-445a-ac87-dca35447093c,raw,9,catalog,2019-11-01 22:29:01.049513,629881394
11,9e74c251-b5d9-41f5-9178-55b869221bcd,raw,11,catalog,2019-11-01 22:29:32.322458,629881394
13,7519f116-53fd-43a5-9408-b39aee3e2e91,raw,13,catalog,2019-11-01 22:30:09.450839,629881394
14,e3efa4db-637e-4f7d-97ca-47627452e66e,raw,14,catalog,2019-11-01 22:31:05.565762,629881394
15,33fd2fec-0669-4448-b679-a61166ddeed5,raw,15,main,2019-11-01 22:31:08.333560,629881394
16,13a72e04-a34e-4efc-b2bb-2b280c76498d,raw,16,catalog,2019-11-01 22:31:09.010626,629881394
17,4d6b8867-1543-48ed-b9a6-fffdd5f66f57,raw,17,product1,2019-11-01 22:31:10.416231,629881394
18,af71ac16-aca0-41df-b073-4055d2b1951a,raw,18,catalog,2019-11-01 22:31:43.019527,629881394
19,c0935c1a-64af-4b5e-a297-fc9b8ba15c77,raw,19,catalog,2019-11-01 22:32:01.596163,629881394


In [24]:
print(len(source_df[source_df['user_id'] == check_id]))
source_df[source_df['user_id'] == check_id]

48


,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,9863622f-3251-4e1f-a28d-2cfb9889736b,raw,7,main,2019-11-01 22:28:54.791683,629881394
9,3ce21b73-8cd5-445a-ac87-dca35447093c,raw,9,catalog,2019-11-01 22:29:01.049513,629881394
11,9e74c251-b5d9-41f5-9178-55b869221bcd,raw,11,catalog,2019-11-01 22:29:32.322458,629881394
13,7519f116-53fd-43a5-9408-b39aee3e2e91,raw,13,catalog,2019-11-01 22:30:09.450839,629881394
14,e3efa4db-637e-4f7d-97ca-47627452e66e,raw,14,catalog,2019-11-01 22:31:05.565762,629881394
15,33fd2fec-0669-4448-b679-a61166ddeed5,raw,15,main,2019-11-01 22:31:08.333560,629881394
16,13a72e04-a34e-4efc-b2bb-2b280c76498d,raw,16,catalog,2019-11-01 22:31:09.010626,629881394
17,4d6b8867-1543-48ed-b9a6-fffdd5f66f57,raw,17,product1,2019-11-01 22:31:10.416231,629881394
18,af71ac16-aca0-41df-b073-4055d2b1951a,raw,18,catalog,2019-11-01 22:31:43.019527,629881394
19,c0935c1a-64af-4b5e-a297-fc9b8ba15c77,raw,19,catalog,2019-11-01 22:32:01.596163,629881394


User_id = 47562303
Событие встречается дважды


In [25]:
check_id = 47562303
print(len(df81[df81['user_id'] == check_id]))
event_time = source_df[(source_df['user_id'] == check_id) & \
    (source_df['event_name'] == 'payment_done')]['event_timestamp'].min()

6


In [26]:
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] <= event_time)]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
4029,f3cd424d-7e21-425e-b465-8d40553ce09f,raw,4029,catalog,2019-12-29 16:55:27.897130,47562303
4030,e4016cf1-1e50-481e-ad8b-9cb3f90ce812,raw,4030,cart,2019-12-29 16:55:41.539826,47562303
4031,c31a4539-db04-4f50-81b2-1e33ae87d974,raw,4031,delivery_choice,2019-12-29 16:55:44.747151,47562303
4032,632bf62b-15ce-40aa-809d-afbb74ed6790,raw,4032,delivery_courier,2019-12-29 16:55:49.061226,47562303
4033,9819bcbc-ee39-4f05-9ff7-64081cba2977,raw,4033,payment_choice,2019-12-29 16:55:49.201853,47562303
4034,ce4619bc-a4fe-4400-8971-fc943892b5e2,raw,4034,payment_cash,2019-12-29 16:55:52.004250,47562303
4035,b5f21859-def2-4daa-84d5-8804e75dc1e4,raw,4035,payment_done,2019-12-29 16:55:52.350396,47562303


In [27]:
source_df[(source_df['event_name'] == 'payment_done') | \
          (source_df['event_name'] == 'payment_cash') ]\
    .groupby('user_id')['event_name'].nunique().sort_values(ascending=False)

user_id
168688753    2
830731696    2
849341362    2
845778461    2
652347392    2
            ..
355840584    1
358346667    1
363256580    1
365193081    1
999642905    1
Name: event_name, Length: 730, dtype: int64

In [28]:
source_df['event_name'].value_counts()

catalog             14518
main                 5635
lost                 3098
cart                 2842
product2             2172
delivery_choice      1686
product1             1515
payment_choice       1107
delivery_courier      834
payment_done          706
payment_card          565
delivery_pickup       506
payment_cash          197
Name: event_name, dtype: int64

### min_cjm

In [29]:
from src.data_processors_lib.rete import CutPathAfterEvent, CutPathAfterEventParams
params = {
    'cutoff_events': ['payment_done'],
    'cut_shift': 0,
    'min_cjm': 3
}

cut_after = EventsNode(
    CutPathAfterEvent(params=CutPathAfterEventParams(**params)))

graph.add_node(
    node=cut_after,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_after
)

/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_after_event.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut["rw_cumsum"] = df_cut.loc[::-1].groupby([user_col])[time_col].transform(


In [30]:
df82 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df82.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [31]:
df82['event_type'].value_counts()

raw    31780
Name: event_type, dtype: int64

In [32]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [33]:
source_df[source_df['event_name'] == 'payment_done']['user_id'].nunique()

653

In [34]:
df8.groupby(['user_id']).count()['event_id'].value_counts()

2     443
3     380
4     352
7     321
8     297
5     282
6     265
9     221
10    190
11    137
12    123
13    102
14     97
15     70
18     56
16     56
17     47
19     40
20     39
21     32
22     28
23     26
27     18
24     14
25     12
28     11
26     10
32     10
34      9
31      8
35      8
29      6
45      4
36      4
30      4
33      3
51      3
37      3
46      2
49      2
38      2
75      1
43      1
39      1
47      1
55      1
63      1
57      1
48      1
42      1
44      1
50      1
62      1
73      1
85      1
Name: event_id, dtype: int64

In [35]:
df82.groupby(['user_id']).count()['event_id'].value_counts()

3     380
4     352
7     321
8     297
5     282
6     265
9     221
10    190
11    137
12    123
13    102
14     97
15     70
18     56
16     56
17     47
19     40
20     39
21     32
22     28
23     26
27     18
24     14
25     12
28     11
32     10
26     10
34      9
31      8
35      8
29      6
45      4
36      4
30      4
33      3
51      3
37      3
46      2
49      2
38      2
75      1
43      1
39      1
47      1
55      1
63      1
57      1
48      1
42      1
44      1
50      1
62      1
73      1
85      1
Name: event_id, dtype: int64

### Несколько событий

In [36]:
from src.data_processors_lib.rete import CutPathAfterEvent, CutPathAfterEventParams
params = {
    'cutoff_events': ['payment_done', 'payment_cash', 'delivery_choice'],
    'cut_shift': 0,
    'min_cjm': 0
}

cut_after = EventsNode(
    CutPathAfterEvent(params=CutPathAfterEventParams(**params)))

graph.add_node(
    node=cut_after,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_after
)

/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_after_event.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut["rw_cumsum"] = df_cut.loc[::-1].groupby([user_col])[time_col].transform(
/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_after_event.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_del["ref"] = df_to_del[eventstream.schema.event_id]


In [37]:
df83 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df83.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [38]:
check_id = 168688753
print(len(df83[df83['user_id'] == check_id]))
event_time = source_df[(source_df['user_id'] == check_id) & \
    ((source_df['event_name'] == 'payment_done') | \
    (source_df['event_name'] == 'payment_cash') | \
     (source_df['event_name'] == 'delivery_choice')
     )]['event_timestamp'].min()
event_time

14


Timestamp('2019-12-14 06:33:57.636413')

In [39]:
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] <= event_time)]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
842,a9a95b69-4698-4c93-ab8c-2ce6b33663a1,raw,842,catalog,2019-11-16 22:30:05.735414,168688753
843,569bc5ea-db2f-4386-b916-81b32d505768,raw,843,catalog,2019-11-16 22:30:15.425342,168688753
844,5c340007-da4f-4ff5-a31b-3622515b0eef,raw,844,catalog,2019-11-16 22:30:40.187585,168688753
2286,7460e6f4-97fe-4b29-afa3-7f342dca936d,raw,2286,main,2019-12-06 21:40:31.972558,168688753
2833,d06d2b60-2950-4828-be8f-447ef58d2cb0,raw,2833,main,2019-12-14 06:32:50.934854,168688753
2834,9df8289f-17bb-435d-8747-701598f47c9e,raw,2834,catalog,2019-12-14 06:32:56.369665,168688753
2835,be3c28b6-9f54-44cc-ad8d-6ebbf0cd55f0,raw,2835,catalog,2019-12-14 06:33:12.102891,168688753
2836,6525687a-7b66-4f6e-95a7-6821f0c6454c,raw,2836,catalog,2019-12-14 06:33:28.660860,168688753
2837,68e9e510-8c67-409f-a2fa-77a411cdd4b3,raw,2837,product2,2019-12-14 06:33:29.294200,168688753
2838,2a898844-b111-4ba6-b479-fa25db67e7a5,raw,2838,catalog,2019-12-14 06:33:30.657942,168688753


In [40]:
df83[df83['user_id'] == check_id]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
986,a9a95b69-4698-4c93-ab8c-2ce6b33663a1,raw,986,catalog,2019-11-16 22:30:05.735414,168688753.0
987,569bc5ea-db2f-4386-b916-81b32d505768,raw,987,catalog,2019-11-16 22:30:15.425342,168688753.0
988,5c340007-da4f-4ff5-a31b-3622515b0eef,raw,988,catalog,2019-11-16 22:30:40.187585,168688753.0
2859,7460e6f4-97fe-4b29-afa3-7f342dca936d,raw,2859,main,2019-12-06 21:40:31.972558,168688753.0
3535,d06d2b60-2950-4828-be8f-447ef58d2cb0,raw,3535,main,2019-12-14 06:32:50.934854,168688753.0
3536,9df8289f-17bb-435d-8747-701598f47c9e,raw,3536,catalog,2019-12-14 06:32:56.369665,168688753.0
3537,be3c28b6-9f54-44cc-ad8d-6ebbf0cd55f0,raw,3537,catalog,2019-12-14 06:33:12.102891,168688753.0
3538,6525687a-7b66-4f6e-95a7-6821f0c6454c,raw,3538,catalog,2019-12-14 06:33:28.660860,168688753.0
3539,68e9e510-8c67-409f-a2fa-77a411cdd4b3,raw,3539,product2,2019-12-14 06:33:29.294200,168688753.0
3540,2a898844-b111-4ba6-b479-fa25db67e7a5,raw,3540,catalog,2019-12-14 06:33:30.657942,168688753.0


In [41]:
source_df[(source_df['user_id'] == check_id) ]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
842,a9a95b69-4698-4c93-ab8c-2ce6b33663a1,raw,842,catalog,2019-11-16 22:30:05.735414,168688753
843,569bc5ea-db2f-4386-b916-81b32d505768,raw,843,catalog,2019-11-16 22:30:15.425342,168688753
844,5c340007-da4f-4ff5-a31b-3622515b0eef,raw,844,catalog,2019-11-16 22:30:40.187585,168688753
2286,7460e6f4-97fe-4b29-afa3-7f342dca936d,raw,2286,main,2019-12-06 21:40:31.972558,168688753
2833,d06d2b60-2950-4828-be8f-447ef58d2cb0,raw,2833,main,2019-12-14 06:32:50.934854,168688753
2834,9df8289f-17bb-435d-8747-701598f47c9e,raw,2834,catalog,2019-12-14 06:32:56.369665,168688753
2835,be3c28b6-9f54-44cc-ad8d-6ebbf0cd55f0,raw,2835,catalog,2019-12-14 06:33:12.102891,168688753
2836,6525687a-7b66-4f6e-95a7-6821f0c6454c,raw,2836,catalog,2019-12-14 06:33:28.660860,168688753
2837,68e9e510-8c67-409f-a2fa-77a411cdd4b3,raw,2837,product2,2019-12-14 06:33:29.294200,168688753
2838,2a898844-b111-4ba6-b479-fa25db67e7a5,raw,2838,catalog,2019-12-14 06:33:30.657942,168688753


### Результат

1) [TODO] Ворнинги
2) [DONE] Не работает обрезка
3) [TODO] Медленная работа - оптимизация кода?
4) [DONE] Неправильная работа, если задать несколько событий, обрезка происходит по последнему, а не по первому событию
5) [DONE] протестировать сдвиги и минимальные траектории
6) [DONE] неправильная работа soft_delete и создание ивентстрима (при копировании не пробрасываются старые айдишники у ивентов, а создаются новые, поэтому невозможно применить soft_delete к новому eventstream)

# SplitSessions

### Описание работы функции

Функция разделяет траектории пользователя не сессии
По заданному порогу ```session_cutoff``` добавляет в eventstream события session_start и session_end

Если между двумя соседними событиями одного пользователя проходит больше, чем ```session_cutoff``` времени, то добавляет события session_end c timestamp последнего события в этом периоде и session_start. При этом добавляется колонка session_id со значениями вида user_id_session_id. Пороговое значение можно определить на глаз с помощью гистограммы ```preprocessing.plot_time_delta_between_2_events```

Если установлен параметр - mark_truncated=False
То сравнивается время 1 события всего датасета и время 1 события каждого пользователя (и также последнее событие в траектории каждого пользователя с последним событием в датасете).
Если полученное время меньше, чем заданная отсечка, то мы считаем сессию разорванной и из нее удаляются все события кроме: старта/окончания траектории и:
- Start и session_end (если разорвана слева)
- End и session_start (если разорвана справа)


In [8]:
from src.data_processors_lib.rete import SplitSessions, SplitSessionsParams
params = {
    'session_cutoff': (1000, 's'),
    # 'mark_truncated': ,
    'session_col': 'session_id'
}

split_sessions = EventsNode(
    SplitSessions(params=SplitSessionsParams(**params)))

graph.add_node(
    node=split_sessions,
    parents=[graph.root]
)

result = graph.combine(
    node=split_sessions
)


In [9]:
df10 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df10.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [10]:
df10.head()

,event_id,event_type,event_index,event_name,event_timestamp,user_id,session_id
0,9cfad64e-0190-4d9a-8a66-b19e8f75da45,session_start,0,session_start,2019-11-01 17:59:13.273932,219483890.0,219483890_0
1,dec4c9c8-322e-48c0-9771-4abe32065f82,raw,1,catalog,2019-11-01 17:59:13.273932,219483890.0,219483890_0
3,57fdc854-78eb-4886-a58a-feca5d52cce4,raw,3,product1,2019-11-01 17:59:28.459271,219483890.0,219483890_0
5,0f3eccee-71df-4b11-a402-852ad80fd5fa,raw,5,cart,2019-11-01 17:59:29.502214,219483890.0,219483890_0
7,7bcfca26-66ac-44bd-ab54-94c0fc2bc35c,raw,7,catalog,2019-11-01 17:59:32.557029,219483890.0,219483890_0


In [11]:
df10['event_type'].value_counts()

raw              35381
session_start     6455
session_end       6455
Name: event_type, dtype: int64

In [8]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [9]:
print(len(df10[df10['user_id'] == 219483890]))
df10[df10['user_id'] == 219483890]

25


,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,e29ad367-5e8f-448c-800b-95bf158e68f6,session_start,0,session_start,2019-11-01 17:59:13.273932,219483890.0
1,97a3d279-cf55-4c8d-ab1f-03ab68d744cb,raw,1,catalog,2019-11-01 17:59:13.273932,219483890.0
3,dd002364-9ccb-4bad-abae-6411908ba90d,raw,3,product1,2019-11-01 17:59:28.459271,219483890.0
5,c473af50-b4bc-4fc9-9094-b922ac133149,raw,5,cart,2019-11-01 17:59:29.502214,219483890.0
7,35a10406-c994-4e01-ba55-68aa410d7b7f,raw,7,catalog,2019-11-01 17:59:32.557029,219483890.0
9,3047487d-5495-4124-b7c0-3c40ff54be16,session_end,9,session_end,2019-11-01 17:59:32.557029,219483890.0
5316,82c34beb-c343-4834-b51e-ac1a036bf1ad,session_start,5316,session_start,2019-12-06 16:22:57.484842,219483890.0
5317,f1af8eec-df09-41fd-bd34-f0e7cc9e1c80,raw,5317,main,2019-12-06 16:22:57.484842,219483890.0
5319,36ed82a1-5e86-41a7-8919-ed13d8971959,raw,5319,catalog,2019-12-06 16:23:01.331109,219483890.0
5321,ccf6bd07-4069-406e-a478-83a0add9f127,raw,5321,catalog,2019-12-06 16:23:48.116617,219483890.0


In [48]:
print(len(source_df[source_df['user_id'] == 219483890]))
source_df[source_df['user_id'] == 219483890]

17


,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,c877e23b-b6ce-4205-a406-48d254f10fcd,raw,0,catalog,2019-11-01 17:59:13.273932,219483890
1,2a4981b5-6743-4354-be5a-cb9ae05370a4,raw,1,product1,2019-11-01 17:59:28.459271,219483890
2,2b56b533-1551-4b91-8666-de51c42ebe77,raw,2,cart,2019-11-01 17:59:29.502214,219483890
3,43f3a11e-1a91-4e44-a800-64288fa37a60,raw,3,catalog,2019-11-01 17:59:32.557029,219483890
2245,eb33dd58-e204-49d2-831f-f1a0d168d643,raw,2245,main,2019-12-06 16:22:57.484842,219483890
2246,0d2e4b90-4d99-4fb0-bf23-00e5a34a1be1,raw,2246,catalog,2019-12-06 16:23:01.331109,219483890
2247,b5f75133-6ee4-4fe3-bb76-198cdcbabe71,raw,2247,catalog,2019-12-06 16:23:48.116617,219483890
4836,ad3ecc26-36f8-49dc-a45a-98de3929fd49,raw,4836,main,2020-01-06 22:10:13.635011,219483890
4837,0f96ff74-03fb-44ee-8eb9-9c9bf6c8ff7a,raw,4837,catalog,2020-01-06 22:10:15.228575,219483890
4838,529d938c-f07c-45d3-bf1b-ccd07ee19cf8,raw,4838,cart,2020-01-06 22:10:42.309028,219483890


In [49]:
source.to_dataframe()

,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,c877e23b-b6ce-4205-a406-48d254f10fcd,raw,0,catalog,2019-11-01 17:59:13.273932,219483890
1,2a4981b5-6743-4354-be5a-cb9ae05370a4,raw,1,product1,2019-11-01 17:59:28.459271,219483890
2,2b56b533-1551-4b91-8666-de51c42ebe77,raw,2,cart,2019-11-01 17:59:29.502214,219483890
3,43f3a11e-1a91-4e44-a800-64288fa37a60,raw,3,catalog,2019-11-01 17:59:32.557029,219483890
4,8d8f00c5-3014-4a3d-8ab2-bc46554fc783,raw,4,catalog,2019-11-01 21:38:19.283663,964964743
...,...,...,...,...,...,...
35376,7d75ea22-bc84-4fed-9828-983679c94492,raw,35376,catalog,2020-04-29 12:47:40.975732,501098384
35377,5e0ab82f-0065-4f62-9f24-f2687b1a2eda,raw,35377,catalog,2020-04-29 12:48:01.809577,501098384
35378,bad842c4-00af-49d0-8ce1-26cb0be9a57d,raw,35378,main,2020-04-29 12:48:01.938488,501098384
35379,01c0b81e-c98e-4369-9482-561ad97120af,raw,35379,catalog,2020-04-29 12:48:06.595390,501098384


## Результат

1) [DONE] Дубликаты
2) [DONE] Некорректно работает параметр "Mark_truncated" - должны удалиться все события, кроме Start и end_session для сессий обрезанных сначала и end и start_session для сессий обрезанных в конце
3) [DONE] Не добавляется колонка session_id
4) [TODO_analytics] более точно проверить разбивку на сессии
5) [TODO] Ворнинги
6) [TODO] Протестить при последовательном добавлении
7) [DONE] неправильная работа soft_delete и создание ивентстрима (при копировании не пробрасываются старые айдишники у ивентов, а создаются новые, поэтому невозможно применить soft_delete к новому eventstream)
8) [TODO_analytics] Продумать - нужен ли параметр - какие типы событий удалять?
9) [TODO_analytics] Нужны ли свои отсечки для прометки truncated?
10) [TODO] Обсудить - как добавляется новая колонка


# CutPathBefore

### Описание работы функции

Обрезает eventstream сразу после одного из событий, указанных в списке cutoff_events (первого в траектории). Остается часть траектории, после первого найденного события из списка включительно.

Параметры:
```cut_shift``` - смещение относительно обозначенного в сторону сокращения  (вправо)
```min_cjm``` - минимальное кол-во событий в траектории, которые надо оставить, если в траектории меньшее количество событий, то они будут удалены


### 1 событие

In [50]:
from src.data_processors_lib.rete import CutPathBeforeEvent, CutPathBeforeEventParams
params = {
    'cutoff_events': ['product2'],
    'cut_shift': 0,
    'min_cjm': 0
}

cut_before = EventsNode(
    CutPathBeforeEvent(params=CutPathBeforeEventParams(**params)))

graph.add_node(
    node=cut_before,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_before
)


/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_before_event.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut["num_groups"] = df_cut.groupby([user_col])[time_col].transform(
/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_before_event.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_del["ref"] = df_to_del[eventstream.schema.event_id]


In [51]:
df91 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df91.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [52]:
df91['event_type'].value_counts()

raw    14669
Name: event_type, dtype: int64

In [53]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [54]:
source_df[source_df['event_name'] == 'product2']['user_id'].nunique()

1430

In [ ]:
check_id = 629881394
# product2 встречается дважды
print('Старая длина траектории - ', len(source_df[source_df['user_id'] == check_id]))
print('Новая длина траектории - ', len(df91[df91['user_id'] == check_id]))

event_time = source_df[(source_df['user_id'] == check_id) & \
    (source_df['event_name'] == 'product2')]['event_timestamp'].min()

In [56]:
df91[df91['user_id'] == check_id]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
44,5acb3cba-b641-43dc-9422-72cecbc435e4,raw,44,product2,2019-11-01 22:32:16.496241,629881394.0
45,4ae55fdc-beb3-42de-834f-1e958153d510,raw,45,catalog,2019-11-01 22:33:17.682468,629881394.0
46,f65ac25a-ae05-4294-aab1-6c57bf7cd4f8,raw,46,product1,2019-11-01 22:33:19.961115,629881394.0
47,43cdf470-32ae-4108-ab22-1942497fe773,raw,47,catalog,2019-11-01 22:33:35.460345,629881394.0
50,f822d087-d4af-4572-bff8-66b49f70f8b3,raw,50,catalog,2019-11-01 22:34:02.301293,629881394.0
53,3f050d42-2416-4ddc-baac-cc4c4a28aa87,raw,53,product2,2019-11-01 22:34:32.362386,629881394.0
55,e11b1664-d137-449d-8910-a642167834f2,raw,55,catalog,2019-11-01 22:35:01.462515,629881394.0
57,b143afc9-c6e9-42a6-afcc-dfe4e82e803e,raw,57,product2,2019-11-01 22:35:33.142711,629881394.0
60,12224283-faa9-410e-9da0-4ed25f431c43,raw,60,cart,2019-11-01 22:35:50.437706,629881394.0
62,91a769a8-9bde-4b98-8312-b753410b48fa,raw,62,delivery_choice,2019-11-01 22:35:57.649549,629881394.0


In [57]:
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] >= event_time)]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
22,5acb3cba-b641-43dc-9422-72cecbc435e4,raw,22,product2,2019-11-01 22:32:16.496241,629881394
23,4ae55fdc-beb3-42de-834f-1e958153d510,raw,23,catalog,2019-11-01 22:33:17.682468,629881394
24,f65ac25a-ae05-4294-aab1-6c57bf7cd4f8,raw,24,product1,2019-11-01 22:33:19.961115,629881394
25,43cdf470-32ae-4108-ab22-1942497fe773,raw,25,catalog,2019-11-01 22:33:35.460345,629881394
27,f822d087-d4af-4572-bff8-66b49f70f8b3,raw,27,catalog,2019-11-01 22:34:02.301293,629881394
29,3f050d42-2416-4ddc-baac-cc4c4a28aa87,raw,29,product2,2019-11-01 22:34:32.362386,629881394
31,e11b1664-d137-449d-8910-a642167834f2,raw,31,catalog,2019-11-01 22:35:01.462515,629881394
33,b143afc9-c6e9-42a6-afcc-dfe4e82e803e,raw,33,product2,2019-11-01 22:35:33.142711,629881394
36,12224283-faa9-410e-9da0-4ed25f431c43,raw,36,cart,2019-11-01 22:35:50.437706,629881394
38,91a769a8-9bde-4b98-8312-b753410b48fa,raw,38,delivery_choice,2019-11-01 22:35:57.649549,629881394


In [58]:
print(len(source_df[source_df['user_id'] == check_id]))
source_df[source_df['user_id'] == check_id]

48


,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,9863622f-3251-4e1f-a28d-2cfb9889736b,raw,7,main,2019-11-01 22:28:54.791683,629881394
9,3ce21b73-8cd5-445a-ac87-dca35447093c,raw,9,catalog,2019-11-01 22:29:01.049513,629881394
11,9e74c251-b5d9-41f5-9178-55b869221bcd,raw,11,catalog,2019-11-01 22:29:32.322458,629881394
13,7519f116-53fd-43a5-9408-b39aee3e2e91,raw,13,catalog,2019-11-01 22:30:09.450839,629881394
14,e3efa4db-637e-4f7d-97ca-47627452e66e,raw,14,catalog,2019-11-01 22:31:05.565762,629881394
15,33fd2fec-0669-4448-b679-a61166ddeed5,raw,15,main,2019-11-01 22:31:08.333560,629881394
16,13a72e04-a34e-4efc-b2bb-2b280c76498d,raw,16,catalog,2019-11-01 22:31:09.010626,629881394
17,4d6b8867-1543-48ed-b9a6-fffdd5f66f57,raw,17,product1,2019-11-01 22:31:10.416231,629881394
18,af71ac16-aca0-41df-b073-4055d2b1951a,raw,18,catalog,2019-11-01 22:31:43.019527,629881394
19,c0935c1a-64af-4b5e-a297-fc9b8ba15c77,raw,19,catalog,2019-11-01 22:32:01.596163,629881394


### сдвиг

In [59]:
from src.data_processors_lib.rete import CutPathBeforeEvent, CutPathBeforeEventParams
params = {
    'cutoff_events': ['product2'],
    'cut_shift': 1,
    'min_cjm': 0
}

cut_before = EventsNode(
    CutPathBeforeEvent(params=CutPathBeforeEventParams(**params)))

graph.add_node(
    node=cut_before,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_before
)


/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_before_event.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut["num_groups"] = df_cut.groupby([user_col])[time_col].transform(
/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_before_event.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_del["ref"] = df_to_del[eventstream.schema.event_id]


In [60]:
df92 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df92.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [61]:
df92['event_type'].value_counts()

raw    13239
Name: event_type, dtype: int64

In [62]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [63]:
source_df[source_df['event_name'] == 'product2']['user_id'].nunique()

1430

In [64]:
check_id = 629881394
print(f'Длина отрезанной траектории для пользователя {check_id} - ',
      len(df92[df92['user_id'] == check_id]))
df92[df92['user_id'] == check_id]

Длина отрезанной траектории для пользователя 629881394 -  35


,event_id,event_type,event_index,event_name,event_timestamp,user_id
46,4ae55fdc-beb3-42de-834f-1e958153d510,raw,46,catalog,2019-11-01 22:33:17.682468,629881394.0
47,f65ac25a-ae05-4294-aab1-6c57bf7cd4f8,raw,47,product1,2019-11-01 22:33:19.961115,629881394.0
48,43cdf470-32ae-4108-ab22-1942497fe773,raw,48,catalog,2019-11-01 22:33:35.460345,629881394.0
51,f822d087-d4af-4572-bff8-66b49f70f8b3,raw,51,catalog,2019-11-01 22:34:02.301293,629881394.0
54,3f050d42-2416-4ddc-baac-cc4c4a28aa87,raw,54,product2,2019-11-01 22:34:32.362386,629881394.0
57,e11b1664-d137-449d-8910-a642167834f2,raw,57,catalog,2019-11-01 22:35:01.462515,629881394.0
59,b143afc9-c6e9-42a6-afcc-dfe4e82e803e,raw,59,product2,2019-11-01 22:35:33.142711,629881394.0
62,12224283-faa9-410e-9da0-4ed25f431c43,raw,62,cart,2019-11-01 22:35:50.437706,629881394.0
64,91a769a8-9bde-4b98-8312-b753410b48fa,raw,64,delivery_choice,2019-11-01 22:35:57.649549,629881394.0
65,d2a1e54e-0780-4678-ae3c-6251110c1b53,raw,65,delivery_courier,2019-11-01 22:36:02.009271,629881394.0


In [65]:
# Время, по которому обрезается траектория, здесь задаем конкретное событие payment_cash, для теста надо будет автоматизировать

event_time = source_df[(source_df['user_id'] == check_id) & \
    (source_df['event_name'] == 'product2')]['event_timestamp'].min()

In [66]:
print(f'Длина обрезанной траектории из source_df для пользователя  {check_id} - ', len(source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] > event_time)]))
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] > event_time)]

Длина обрезанной траектории из source_df для пользователя  629881394 -  35


,event_id,event_type,event_index,event_name,event_timestamp,user_id
23,4ae55fdc-beb3-42de-834f-1e958153d510,raw,23,catalog,2019-11-01 22:33:17.682468,629881394
24,f65ac25a-ae05-4294-aab1-6c57bf7cd4f8,raw,24,product1,2019-11-01 22:33:19.961115,629881394
25,43cdf470-32ae-4108-ab22-1942497fe773,raw,25,catalog,2019-11-01 22:33:35.460345,629881394
27,f822d087-d4af-4572-bff8-66b49f70f8b3,raw,27,catalog,2019-11-01 22:34:02.301293,629881394
29,3f050d42-2416-4ddc-baac-cc4c4a28aa87,raw,29,product2,2019-11-01 22:34:32.362386,629881394
31,e11b1664-d137-449d-8910-a642167834f2,raw,31,catalog,2019-11-01 22:35:01.462515,629881394
33,b143afc9-c6e9-42a6-afcc-dfe4e82e803e,raw,33,product2,2019-11-01 22:35:33.142711,629881394
36,12224283-faa9-410e-9da0-4ed25f431c43,raw,36,cart,2019-11-01 22:35:50.437706,629881394
38,91a769a8-9bde-4b98-8312-b753410b48fa,raw,38,delivery_choice,2019-11-01 22:35:57.649549,629881394
39,d2a1e54e-0780-4678-ae3c-6251110c1b53,raw,39,delivery_courier,2019-11-01 22:36:02.009271,629881394


In [67]:
print(len(source_df[source_df['user_id'] == check_id]))
source_df[source_df['user_id'] == check_id]

48


,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,9863622f-3251-4e1f-a28d-2cfb9889736b,raw,7,main,2019-11-01 22:28:54.791683,629881394
9,3ce21b73-8cd5-445a-ac87-dca35447093c,raw,9,catalog,2019-11-01 22:29:01.049513,629881394
11,9e74c251-b5d9-41f5-9178-55b869221bcd,raw,11,catalog,2019-11-01 22:29:32.322458,629881394
13,7519f116-53fd-43a5-9408-b39aee3e2e91,raw,13,catalog,2019-11-01 22:30:09.450839,629881394
14,e3efa4db-637e-4f7d-97ca-47627452e66e,raw,14,catalog,2019-11-01 22:31:05.565762,629881394
15,33fd2fec-0669-4448-b679-a61166ddeed5,raw,15,main,2019-11-01 22:31:08.333560,629881394
16,13a72e04-a34e-4efc-b2bb-2b280c76498d,raw,16,catalog,2019-11-01 22:31:09.010626,629881394
17,4d6b8867-1543-48ed-b9a6-fffdd5f66f57,raw,17,product1,2019-11-01 22:31:10.416231,629881394
18,af71ac16-aca0-41df-b073-4055d2b1951a,raw,18,catalog,2019-11-01 22:31:43.019527,629881394
19,c0935c1a-64af-4b5e-a297-fc9b8ba15c77,raw,19,catalog,2019-11-01 22:32:01.596163,629881394


### min_cjm

In [68]:
from src.data_processors_lib.rete import CutPathBeforeEvent, CutPathBeforeEventParams
params = {
    'cutoff_events': ['product2'],
    'cut_shift': 0,
    'min_cjm': 3
}

cut_before = EventsNode(
    CutPathBeforeEvent(params=CutPathBeforeEventParams(**params)))

graph.add_node(
    node=cut_before,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_before
)


/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_before_event.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut["num_groups"] = df_cut.groupby([user_col])[time_col].transform(


NameError: name 'pd' is not defined

In [ ]:
df93 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df93.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

In [ ]:
df93['event_type'].value_counts()

In [ ]:
source_df['event_type'].value_counts()

In [ ]:
df91.groupby(['user_id']).count()['event_id'].value_counts().sort_index()

In [ ]:
df93.groupby(['user_id']).count()['event_id'].value_counts().sort_index()

### Несколько событий

In [ ]:
from src.data_processors_lib.rete import CutPathBeforeEvent, CutPathBeforeEventParams
params = {
    'cutoff_events': ['product2', 'product1'],
    'cut_shift': 0,
    'min_cjm': 0
}

cut_before = EventsNode(
    CutPathBeforeEvent(params=CutPathBeforeEventParams(**params)))

graph.add_node(
    node=cut_before,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_before
)


In [ ]:
df94 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df94.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

In [ ]:
check_id = 168688753
print(len(df94[df94['user_id'] == check_id]))
event_time = source_df[(source_df['user_id'] == check_id) & \
    ((source_df['event_name'] == 'product1') | \
    (source_df['event_name'] == 'product2')
     )]['event_timestamp'].min()
event_time

In [ ]:
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] >= event_time)]

In [ ]:
df94[df94['user_id'] == check_id]

In [ ]:
source_df[(source_df['user_id'] == check_id) ]

### Результат

1) [TODO] Ворнинги
2) [DONE] Не работает обрезка
3) [TODO] Медленная работа - оптимизация кода?
4) [DONE] Неправильная работа, если задать несколько событий, обрезка происходит по последнему, а не по первому событию
5) [DONE] протестировать сдвиги и минимальные траектории
6) [DONE] неправильная работа soft_delete и создание eventstream (при копировании не пробрасываются старые айдишники у ивентов, а создаются новые, поэтому невозможно применить soft_delete к новому eventstream)